# Data Wrangling 

- Shows the use of Pandas library for data analysis and manipulation
- Preparing Data for Feature Engineering
- Removing Outliers and Scaling Data
- These processes will NOT damage your original data if you save changes to a new data file

Loading Dependencies

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy

Storing Data as local variable (pandas object)

In [2]:
data_frame = pd.read_csv("vgsales.csv")

### Handling Null Values

- Dealing will null values will stop them from causing runtime errors or unexpected results
- Dealing with them appropiately will ensure no biases in the data from this correction

Returning the number of null values for each column

In [3]:
data_frame.isnull().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

##### Evaluation
- there is a considerable amount of data points missing year data the key feature in the model
- there is also a some games lacking publishers another features being incoporated in the model
- need to find where the data is located to ensure there isn't bias created from removing the data

showing the data entries with null values:

In [4]:

null_mask = data_frame.isnull().any(axis=1)
null_rows = data_frame[null_mask]
print(null_rows)


        Rank                             Name Platform    Year         Genre  \
179      180                  Madden NFL 2004      PS2     NaN        Sports   
377      378                 FIFA Soccer 2004      PS2     NaN        Sports   
431      432       LEGO Batman: The Videogame      Wii     NaN        Action   
470      471       wwe Smackdown vs. Raw 2006      PS2     NaN      Fighting   
607      608                   Space Invaders     2600     NaN       Shooter   
...      ...                              ...      ...     ...           ...   
16427  16430                     Virtua Quest       GC     NaN  Role-Playing   
16493  16496                       The Smurfs      3DS     NaN        Action   
16494  16497  Legends of Oz: Dorothy's Return      3DS  2014.0        Puzzle   
16543  16546           Driving Simulator 2011       PC  2011.0        Racing   
16553  16556                   Bound By Flame     X360  2014.0  Role-Playing   

                                    Pub

In [5]:
nullist = null_rows.Rank
print(nullist)
print(numpy.std(nullist))

179        180
377        378
431        432
470        471
607        608
         ...  
16427    16430
16493    16496
16494    16497
16543    16546
16553    16556
Name: Rank, Length: 307, dtype: int64
4716.305524906458


Decision/Evaluation:
- The null data is spread out far enough and evenly enough to justify just removing the data without years and publishers
- I noticed that there are the same game on different consoles as different data points

In [6]:
data_frame = data_frame.dropna(subset=['Publisher'])
data_frame.isnull().sum()

Rank              0
Name              0
Platform          0
Year            249
Genre             0
Publisher         0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

Years will be treated as more categorical data rather than numerical. This will avoid the common practice of taking the average of the column for that data and all the potential biases and skews this could cause when applying it to years. This data will just be removed for this model since it is a small amount of data.

In [7]:
data_frame = data_frame.dropna(subset=['Year'])
data_frame.isnull().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

### Duplicates

Duplicate Data will have significant effects on the model and often lead to over or under fitting in the resulting model. Deleting these points will avoid this

In [8]:
data_frame.duplicated().sum()

np.int64(0)

There is no duplicate data so this process is complete.

### Remove Outliers

Outlier will skew data. This remove them, the use of the statistical math definition of outliers will work. Find the IQR by finding Quartile 1 and Quartile 3 and then formula for finding outliers will be applied to it where it is 1.5 * IQR above or below Q1 or Q3 respectively.

Global_Sales Outliers Removal

In [9]:
print(data_frame['Global_Sales'].describe())
Q1 = data_frame['Global_Sales'].quantile(0.25)
Q3 = data_frame['Global_Sales'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a Global_Sales above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')

count    16291.000000
mean         0.540910
std          1.567345
min          0.010000
25%          0.060000
50%          0.170000
75%          0.480000
max         82.740000
Name: Global_Sales, dtype: float64
Outliers are a Global_Sales above 1.1099999999999999 or below -0.5700000000000001


Removing the outliers

In [10]:
data_frame = data_frame[(data_frame['Global_Sales'] >= Q1 - 1.5 * IQR) & (data_frame['Global_Sales'] <= Q3 + 1.5 * IQR)]
print(data_frame['Global_Sales'].describe())

count    14465.000000
mean         0.234321
std          0.246890
min          0.010000
25%          0.050000
50%          0.140000
75%          0.330000
max          1.100000
Name: Global_Sales, dtype: float64


Removing outliers for all other sales

In [11]:
print(data_frame['NA_Sales'].describe())
Q1 = data_frame['NA_Sales'].quantile(0.25)
Q3 = data_frame['NA_Sales'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a NA_Sales above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')
data_frame = data_frame[(data_frame['NA_Sales'] >= Q1 - 1.5 * IQR) & (data_frame['NA_Sales'] <= Q3 + 1.5 * IQR)]
print(data_frame['NA_Sales'].describe())

count    14465.000000
mean         0.116525
std          0.154818
min          0.000000
25%          0.000000
50%          0.060000
75%          0.160000
max          1.030000
Name: NA_Sales, dtype: float64
Outliers are a NA_Sales above 0.4 or below -0.24
count    13529.000000
mean         0.086277
std          0.101064
min          0.000000
25%          0.000000
50%          0.050000
75%          0.140000
max          0.400000
Name: NA_Sales, dtype: float64


In [12]:
print(data_frame['EU_Sales'].describe())
Q1 = data_frame['EU_Sales'].quantile(0.25)
Q3 = data_frame['EU_Sales'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a EU_Sales above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')
data_frame = data_frame[(data_frame['EU_Sales'] >= Q1 - 1.5 * IQR) & (data_frame['EU_Sales'] <= Q3 + 1.5 * IQR)]
print(data_frame['EU_Sales'].describe())

count    13529.000000
mean         0.050366
std          0.090539
min          0.000000
25%          0.000000
50%          0.010000
75%          0.060000
max          0.980000
Name: EU_Sales, dtype: float64
Outliers are a EU_Sales above 0.15 or below -0.09
count    12222.000000
mean         0.026486
std          0.037025
min          0.000000
25%          0.000000
50%          0.010000
75%          0.040000
max          0.150000
Name: EU_Sales, dtype: float64


In [13]:
print(data_frame['JP_Sales'].describe())
Q1 = data_frame['JP_Sales'].quantile(0.25)
Q3 = data_frame['JP_Sales'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a JP_Sales above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')
data_frame = data_frame[(data_frame['JP_Sales'] >= Q1 - 1.5 * IQR) & (data_frame['JP_Sales'] <= Q3 + 1.5 * IQR)]
print(data_frame['JP_Sales'].describe())

count    12222.000000
mean         0.044789
std          0.110776
min          0.000000
25%          0.000000
50%          0.000000
75%          0.030000
max          1.100000
Name: JP_Sales, dtype: float64
Outliers are a JP_Sales above 0.075 or below -0.045
count    10229.000000
mean         0.007943
std          0.016881
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.070000
Name: JP_Sales, dtype: float64


In [14]:
print(data_frame['Other_Sales'].describe())
Q1 = data_frame['Other_Sales'].quantile(0.25)
Q3 = data_frame['Other_Sales'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a Other_Sales above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')
data_frame = data_frame[(data_frame['Other_Sales'] >= Q1 - 1.5 * IQR) & (data_frame['Other_Sales'] <= Q3 + 1.5 * IQR)]
print(data_frame['Other_Sales'].describe())

count    10229.000000
mean         0.010789
std          0.025882
min          0.000000
25%          0.000000
50%          0.010000
75%          0.010000
max          0.740000
Name: Other_Sales, dtype: float64
Outliers are a Other_Sales above 0.025 or below -0.015
count    9092.000000
mean        0.005863
std         0.007105
min         0.000000
25%         0.000000
50%         0.000000
75%         0.010000
max         0.020000
Name: Other_Sales, dtype: float64


#### Scaling Features to Common Range
scaling features to a set range makes it much easier for all ML algorithms as differing scales wouldn't end up effecting the process

The choose to use 1.2 to scale Global_Sales is based on the data set after removing outliers. This is done to keep the proportions between data columns the same. With doing another trial with independent column scaling the model became much more inaccurate leading to the unified scaling of all sales.

In [15]:
scale_feature = 'Global_Sales'

#the minimum value with space for outliers
MIN_GS = 0

#the maximum value with space for outliers
MAX_GS = 1.2

#scale features
data_frame[scale_feature] = [(X - MIN_GS) / (MAX_GS - MIN_GS) for X in data_frame[scale_feature]]

data_frame.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000
mean,11545.108007,2006.881764,0.066831,0.021737,0.008359,0.005863,0.086168
std,3223.891366,5.187581,0.079106,0.028975,0.017207,0.007105,0.079111
min,3565.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.008333
25%,9007.750000,2004.000000,0.000000,0.000000,0.000000,0.000000,0.025000
50%,11955.500000,2008.000000,0.040000,0.010000,0.000000,0.000000,0.058333
75%,14269.250000,2010.000000,0.100000,0.030000,0.010000,0.010000,0.116667
max,16600.000000,2020.000000,0.400000,0.150000,0.070000,0.020000,0.466667


In [ ]:
scale_feature = 'NA_Sales'

#the minimum value with space for outliers
MIN_NS = 0

#the maximum value with space for outliers
MAX_NS = 1.2

#scale features
data_frame[scale_feature] = [(X - MIN_NS) / (MAX_NS - MIN_NS) for X in data_frame[scale_feature]]

data_frame.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000
mean,11545.108007,2006.881764,0.133663,0.021737,0.008359,0.005863,0.086168
std,3223.891366,5.187581,0.158211,0.028975,0.017207,0.007105,0.079111
min,3565.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.008333
25%,9007.750000,2004.000000,0.000000,0.000000,0.000000,0.000000,0.025000
50%,11955.500000,2008.000000,0.080000,0.010000,0.000000,0.000000,0.058333
75%,14269.250000,2010.000000,0.200000,0.030000,0.010000,0.010000,0.116667
max,16600.000000,2020.000000,0.800000,0.150000,0.070000,0.020000,0.466667


In [ ]:
scale_feature = 'EU_Sales'

#the minimum value with space for outliers
MIN_ES = 0

#the maximum value with space for outliers
MAX_ES = 1.2

#scale features
data_frame[scale_feature] = [(X - MIN_ES) / (MAX_ES - MIN_ES) for X in data_frame[scale_feature]]

data_frame.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000
mean,11545.108007,2006.881764,0.133663,0.108683,0.008359,0.005863,0.086168
std,3223.891366,5.187581,0.158211,0.144876,0.017207,0.007105,0.079111
min,3565.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.008333
25%,9007.750000,2004.000000,0.000000,0.000000,0.000000,0.000000,0.025000
50%,11955.500000,2008.000000,0.080000,0.050000,0.000000,0.000000,0.058333
75%,14269.250000,2010.000000,0.200000,0.150000,0.010000,0.010000,0.116667
max,16600.000000,2020.000000,0.800000,0.750000,0.070000,0.020000,0.466667


In [ ]:
scale_feature = 'JP_Sales'

#the minimum value with space for outliers
MIN_JS = 0

#the maximum value with space for outliers
MAX_JS = 1.2

#scale features
data_frame[scale_feature] = [(X - MIN_JS) / (MAX_JS - MIN_JS) for X in data_frame[scale_feature]]

data_frame.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000
mean,11545.108007,2006.881764,0.133663,0.108683,0.083590,0.005863,0.086168
std,3223.891366,5.187581,0.158211,0.144876,0.172065,0.007105,0.079111
min,3565.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.008333
25%,9007.750000,2004.000000,0.000000,0.000000,0.000000,0.000000,0.025000
50%,11955.500000,2008.000000,0.080000,0.050000,0.000000,0.000000,0.058333
75%,14269.250000,2010.000000,0.200000,0.150000,0.100000,0.010000,0.116667
max,16600.000000,2020.000000,0.800000,0.750000,0.700000,0.020000,0.466667


In [ ]:
scale_feature = 'Other_Sales'

#the minimum value with space for outliers
MIN_OS = 0

#the maximum value with space for outliers
MAX_OS = 1.2

#scale features
data_frame[scale_feature] = [(X - MIN_OS) / (MAX_OS - MIN_OS) for X in data_frame[scale_feature]]

data_frame.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000,9092.000000
mean,11545.108007,2006.881764,0.133663,0.108683,0.083590,0.117268,0.086168
std,3223.891366,5.187581,0.158211,0.144876,0.172065,0.142100,0.079111
min,3565.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.008333
25%,9007.750000,2004.000000,0.000000,0.000000,0.000000,0.000000,0.025000
50%,11955.500000,2008.000000,0.080000,0.050000,0.000000,0.000000,0.058333
75%,14269.250000,2010.000000,0.200000,0.150000,0.100000,0.200000,0.116667
max,16600.000000,2020.000000,0.800000,0.750000,0.700000,0.400000,0.466667


In [ ]:
data_frame.to_csv('../2.Feature_Engineering/wrangled_data.csv', index=False)